In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch
import random

from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

#Pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


#### 1. Load the data - User inputs 'data_type' 

In [5]:
# Potential to remove this and do processing in one go upfront then just load the data as necessary

In [4]:
data_type = 'N1' # Remove this line for script

In [3]:
t1 = time.time()


channels = constants.channel_list

# 1. generate all path names and class list(s) etc. 
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
paths = joblib.load(folder + data_type + '_paths.pkl') # keys : ['selected_paths', 's_class_list', 's_night_list', 's_sleep_type', 's_p_id']

# 2. Load corresponding data into dataframes, store in dataframe list
df_list = []
with io.capture_output() as captured:
    for path in paths['selected_paths']:
        data_epo = mne.read_epochs(path)
        data = data_epo._data * 1e6  # convert signal from V to uV
        df_full = data_epo.to_data_frame()
        df = df_full[channels].copy()
        
        # Do z_score normalisation on the data
        # Got it from here : https://www.geeksforgeeks.org/data-normalization-with-pandas/
        # z_score normalisation __________________
        # copy the data 
        df_z_scaled = df.copy() 

        # apply normalization techniques 
        for column in df_z_scaled.columns: 
            df_z_scaled[column] = (df_z_scaled[column] - df_z_scaled[column].mean()) / df_z_scaled[column].std()

    
        df_list.append(df_z_scaled)


t2 = time.time()

t2-t1

4.689120292663574

In [6]:
class_label_dict = {'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3} #Dictionary used to label the classes for reference
y = np.array([class_label_dict[c_name] for c_name in paths['s_class_list'] ] )
groups = paths['s_p_id']

In [7]:
wake_dfs_binary = []
y_binary = []
groups_binary = []

for df , class_label , group in zip(df_list, y, groups):
    if class_label in [0,1]:
        wake_dfs_binary.append(df)
        y_binary.append(class_label)
        groups_binary.append(group)

y_binary = np.array(y_binary)